In [1]:
# import packages
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, average_precision_score

In [4]:
train = pd.read_excel("lendingclub_traindata.xlsx")
print(train.head())
print("-------------------------------------------")

validation = pd.read_excel("lendingclub_valdata.xlsx")
validation.rename(columns{}
print(validation.head())
print("-------------------------------------------")

test = pd.read_excel("lendingclub_testdata.xlsx")
print(test.head())


   home_ownership    income    dti  fico  loan_status
0               1   44304.0  18.47   690            0
1               0   50000.0  29.62   735            1
2               0   64400.0  16.68   675            1
3               0   38500.0  33.73   660            0
4               1  118000.0  26.66   665            1
-------------------------------------------
   homw_ownership    income    dti  fico  loan_status
0               0   25000.0  27.60   660            0
1               0   50000.0  21.51   715            1
2               1  100000.0   8.14   770            1
3               0   75000.0   1.76   685            0
4               1   78000.0  16.11   680            1
-------------------------------------------
   home_ownership    income    dti  fico  loan_status
0               1   52400.0  24.64   665            1
1               1  150000.0  17.04   785            1
2               1  100000.0  20.92   710            1
3               0   97000.0  13.11   705        

In [7]:
x_train = train.drop('loan_status',axis=1)
x_val = validation.drop('loan_status',axis=1)
x_test = test.drop('loan_status',axis=1)

x_testScale = (x_test-x_train.mean())/x_train.std()
x_valScale = (x_val-x_train.mean())/x_train.std()
x_trainScale = (x_train-x_train.mean())/x_train.std()

y_train = train['loan_status']
y_val = validation['loan_status']
y_test = test['loan_status']

In [11]:
lgstc_reg = LogisticRegression(penalty=None,solver="newton-cg")
lgstc_reg.fit(x_train, y_train)
print(lgstc_reg.intercept_, lgstc_reg.coef_)

[-6.03898462] [[ 2.97143081e-01  6.19442465e-07 -3.65107852e-02  1.13438308e-02]]


In [15]:
lgstc_regScale = LogisticRegression(penalty=None,solver="newton-cg")
lgstc_regScale.fit(x_trainScale, y_train)
print(lgstc_regScale.intercept_, lgstc_regScale.coef_)

[1.41622043] [[ 0.14529381  0.03361951 -0.32404237  0.363174  ]]


In [16]:
y_trainScale_pred = lgstc_regScale.predict_proba(x_trainScale)
y_valScale_pred = lgstc_regScale.predict_proba(x_valScale)
y_test_pred = lgstc_regScale.predict_proba(x_testScale)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- homw_ownership
